In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from process_czi import get_czi_info, chunk_and_save_czi, get_channel_info, calculate_chunks
from io_images import get_images_infoframe
import czifile
import os
import aicspylibczi
import numpy as np


In [ ]:
import platform
print(platform.architecture())

Folder Path

In [ ]:
# Load the .czi file
# folder_path = "/run/user/1001/gvfs/smb-share:server=fs.ista.ac.at,share=drives/tnegrell/group/siegegrp/FlSu/_Bioimaging/imaging files_temp/test_AI14_notam_retina-01_AcquisitionBlock1.czi"

In [ ]:
# Brain Slices
# folder_path = 'data'
folder_path = "/run/user/1001/gvfs/smb-share:server=fs.ista.ac.at,share=drives/tnegrell/archive/siegegrp/AlVe/MORPHOMICS2.0_MICROGLIA_BRAIN_ATLAS"


List of Files in Folder

In [ ]:
# List all files and directories in the folder
contents = os.listdir(folder_path)

print("Contents of the folder:")
for item in contents:
    print(item)

Get Info Frame

In [ ]:
# infoframe = get_images_infoframe(folder_path, conditions=['Animal'])
infoframe = get_images_infoframe(folder_path)

In [ ]:
infoframe

Select one file .czi

In [ ]:
czi_path = infoframe.iloc[5]['file_path']
czi_path

Get Channel Info

In [ ]:
get_channel_info(czi_path)


Get Dim and Size info

In [ ]:
get_czi_info(czi_path)

Chunk dims

In [ ]:
if isinstance(czi_path, str):
    czi_file = aicspylibczi.CziFile(czi_path)
dims = czi_file.get_dims_shape()
chunk_dim_list = []
for dim in dims:
    chunk_dim = calculate_chunks(dim, 30, bytes_per_pixel=2)
    chunk_dim_list.append(chunk_dim)
print(chunk_dim_list)

Test bbox methods of the library

In [ ]:
bbox1 = czi_file.get_all_scene_bounding_boxes()
print(bbox1)
print(bbox1[0].x, bbox1[0].y, bbox1[0].w, bbox1[0].h)
print(bbox1[1].x, bbox1[1].y, bbox1[1].w, bbox1[1].h)

In [ ]:
bbox1[0].x

In [ ]:
# bbox1 = czi_file.get_mosaic_tile_bounding_box() #axis param
print(bbox1[0].x, bbox1[0].y, bbox1[0].w, bbox1[0].h)
print(bbox1[1].x, bbox1[1].y, bbox1[1].w, bbox1[1].h)

In [ ]:
# bbox1 = czi_file.get_tile_bounding_box(C=1, S=0) #axis param
bbox1
# print(bbox1[0].x, bbox1[0].y, bbox1[0].w, bbox1[0].h)

In [ ]:
czi_file.pixel_type

In [ ]:
a = czi_file.read_mosaic(region = (-159614, 25683, 22286, 100), scale_factor=1.0, C=1, Z=0)

In [ ]:
b = czi_file.read_mosaic(region = (-157614, 30683, 200, 200), scale_factor=1.0, C=1)

In [ ]:
a.shape

In [ ]:
# read_subblock_metadata()

Test chunk and save

In [ ]:
import plotly.express as px
import numpy as np


In [ ]:
image = a[0][0]
# for image in data:
# Plot the image
fig = px.imshow(image, color_continuous_scale="gray")
fig.update_layout(title="2D Image", coloraxis_showscale=False)  # Hide color bar
fig.show()

In [ ]:
image = b[0][0]
# for image in data:
# Plot the image
fig = px.imshow(image, color_continuous_scale="gray")
fig.update_layout(title="2D Image", coloraxis_showscale=False)  # Hide color bar
fig.show()

In [ ]:
for m in range(0,3):
    image_data = czi_file.read_image(C = 1, 
                                        S = 0,
                                        M=m,
                                        z=25)
    arr = image_data[0]
    data = arr[0,0,0,0,:,:]
    # for image in data:
    # Plot the image
    fig = px.imshow(data, color_continuous_scale="gray")
    fig.update_layout(title="2D Image", coloraxis_showscale=False)  # Hide color bar
    fig.show()
                                            # dtype = pixel_type,
                                            # **kwargs)

In [ ]:
arr = image_data[0]
arr.shape

In [ ]:
data = arr[0,0,25:30,0,:,:]

In [ ]:
data.shape

In [ ]:
image_data[1]

In [ ]:
my_image = data[2]

In [ ]:
# for image in data:
# Plot the image
fig = px.imshow(my_image, color_continuous_scale="gray")
fig.update_layout(title="2D Image", coloraxis_showscale=False)  # Hide color bar
fig.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go
from PIL import Image
import io
import base64

# Helper function to convert a NumPy array to a base64-encoded image
def array_to_base64(img_array):
    img = Image.fromarray(img_array)  # Convert to PIL Image
    buffer = io.BytesIO()
    img.save(buffer, format="PNG")  # Save image to buffer
    buffer.seek(0)
    encoded_image = base64.b64encode(buffer.read()).decode('utf-8')  # Encode as base64
    return f"data:image/png;base64,{encoded_image}"

# Simulate a 3D image of shape (Z, X, Y)

# Create frames for each Z slice
frames = [
    go.Frame(
        data=[go.Image(source=array_to_base64(data[z]))],  # Convert slice to base64
        name=str(z)  # Frame name is the slice index
    )
    for z in range(data.shape[0])
]

# Initial figure setup with the first slice
fig = go.Figure(
    data=[go.Image(source=array_to_base64(data[0]))],  # Start with the first slice
    frames=frames  # Add all frames
)

# Add slider to navigate through slices
fig.update_layout(
    title="Scroll Through Z-Slices",
    sliders=[
        {
            "steps": [
                {
                    "args": [[str(z)], {"frame": {"duration": 0, "redraw": True}}],
                    "label": f"Z={z}",
                    "method": "animate",
                }
                for z in range(data.shape[0])
            ],
            "currentvalue": {"prefix": "Z-Slice: "},
        }
    ],
    updatemenus=[
        {
            "type": "buttons",
            "showactive": False,
            "buttons": [
                {
                    "label": "Play",
                    "method": "animate",
                    "args": [None, {"frame": {"duration": 200, "redraw": True}, "fromcurrent": True}],
                },
                {
                    "label": "Pause",
                    "method": "animate",
                    "args": [[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate"}],
                },
            ],
        }
    ]
)

fig.show()


In [ ]:
# chunk_and_save_czi(czi_path, 'results', chunk_size = 30, channel_name='EGFP')